### Data generation code : Read the data from file and push the data to producer

##### Import the necessary libraries

In [161]:
import pandas as pd
import datetime as dt
from confluent_kafka import SerializingProducer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.serialization import StringSerializer

In [162]:
def main():
    producer=connect_kafka()
    generate_data_and_publish(producer)
       
    
        

In [163]:
def connect_kafka():
    """
    Connect to kafka and pass the connection details to the next function which will generate the data to push to producer
    """
    kafka_config={
        'bootstrap.servers':'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092',
        'sasl.mechanisms':'PLAIN',
        'security.protocol':'SASL_SSL',
        'sasl.username':'FSBBZWNDP5EGFBXB',
        'sasl.password':'2lSLJIA5hlS5eTareTkxRbaaTi4h//NlNygl6PGyM5vpTIsXBrRY+f/mz1gtuNDT'
    }

    #connect to the schema registry to fetch the required schema details

    schema_registry_client=SchemaRegistryClient({
        'url':'https://psrc-gqrvzv.southeastasia.azure.confluent.cloud',
        'basic.auth.user.info':'{}:{}'.format('NWIGQODEMAQLNVHG', 'srLL0tbfwYYRjsF6LvawiZArZXVYuF1g9/rQWGzi4qz2Zm3kBrUwlX0vinzsyqjH')
        })
    #fetch the latest version of schema registry
    subject_name='logistics_data-value'
    schema_str=schema_registry_client.get_latest_version(subject_name).schema.schema_str

    string_serializer=StringSerializer('UTF-8')
    avro_serializer=AvroSerializer(schema_registry_client,schema_str)

    producer=SerializingProducer({
        'bootstrap.servers':kafka_config['bootstrap.servers'],
        'security.protocol':kafka_config['security.protocol'],
        'sasl.mechanisms':kafka_config['sasl.mechanisms'],
        'sasl.username':kafka_config['sasl.username'],
        'sasl.password':kafka_config['sasl.password'],
        'key.serializer':string_serializer,
        'value.serializer':avro_serializer
    })

    return producer
    
    

##### Function which will take producer configuration as input , generate the data and publish to producer topic

In [164]:
def generate_data_and_publish(producer):
    data1=pd.read_csv(r'C:\Users\siddhant.rawat\Desktop\GrowDataSkills-Studymaterial\MongoDB\Class_Assignment-1-mongo\Class_Assignment\delivery_trip_truck_data.csv')
    #data1=data.rename(columns={"":"Market_Regular"})
    display(data1.head(5))

    for index,row in data1.head(5).iterrows():
        js=row.to_dict()
        #print(js)
        producer.produce(topic='logistics_data',key=str(index),value=js,on_delivery=delivery_report)
        print("published the record with bookingID : {} at index : {} to kafka".format(row["BookingID"],index))
    producer.flush()
    #producer.close()

##### This is delivery report function who function is to throw the error message if an error is encountered and success message if a record is processed successfully

In [165]:
def delivery_report(err,msg):
     if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
     print('User record {} successfully produced to {} [{}] at offset {}'.format(
            msg.key(), msg.topic(), msg.partition(), msg.offset()))
    

In [166]:
if __name__=='__main__':
    main()

,GpsProvider,BookingID,Market_Regular,BookingID_Date,vehicle_no,Origin_Location,Destination_Location,Org_lat_lon,Des_lat_lon,Data_Ping_time,...,TRANSPORTATION_DISTANCE_IN_KM,vehicleType,Minimum_kms_to_be_covered_in_a_day,Driver_Name,Driver_MobileNo,customerID,customerNameCode,supplierID,supplierNameCode,Material_Shipped
0,CONSENT TRACK,MVCV0000927/082021,Market,8/17/2020,KA590408,"TVSLSL-PUZHAL-HUB,CHENNAI,TAMIL NADU","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA","13.1550,80.1960","12.7400,77.8200",05:09.0,...,320.0,NaN,NaN,NaN,NaN,ALLEXCHE45,Ashok leyland limited,VIJEXHOSR7,VIJAY TRANSPORT,BRACKET / GRAB HANDLE
1,VAMOSYS,VCV00014271/082021,Regular,8/27/2020,TN30BC5917,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",40:28.0,...,103.0,NaN,NaN,RAMESH,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,VJLEXSHE09,VJ LOGISTICS,ZB MODEL PLATE / 3143
2,CONSENT TRACK,VCV00014382/082021,Regular,8/27/2020,TN22AR2748,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8710,79.7390","11.8710,79.7390",05:09.0,...,300.0,NaN,NaN,GIRI,NaN,LUTGCCHE06,Lucas tvs ltd,GSTEXLAK1Q,G.S. TRANSPORT,LETTERING / FUSO
3,VAMOSYS,VCV00014743/082021,Regular,8/28/2020,TN28AQ0781,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",40:31.0,...,61.0,NaN,NaN,RAVI,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,ARVEXNAM09,ARVINTH TRANSPORT,LU STRUT RA / RADIUS ROD
4,VAMOSYS,VCV00014744/082021,Regular,8/28/2020,TN68F1722,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8720,79.6320","11.8720,79.6320",40:29.0,...,240.0,NaN,NaN,TAMIL,NaN,LUTGCCHE06,Lucas tvs ltd,SRTEXKOR96,SR TRANSPORTS,WISHBONE / V ROD/HDT


published the record with bookingID : MVCV0000927/082021 at index : 0 to kafka
published the record with bookingID : VCV00014271/082021 at index : 1 to kafka
published the record with bookingID : VCV00014382/082021 at index : 2 to kafka
published the record with bookingID : VCV00014743/082021 at index : 3 to kafka
published the record with bookingID : VCV00014744/082021 at index : 4 to kafka
User record b'4' successfully produced to logistics_data [0] at offset 1
User record b'0' successfully produced to logistics_data [1] at offset 5
User record b'1' successfully produced to logistics_data [1] at offset 6
User record b'2' successfully produced to logistics_data [1] at offset 7
User record b'3' successfully produced to logistics_data [1] at offset 8
